# 5.0.0. Classifier Model with Feature Selection

In [7]:
import pandas as pd
import numpy as np
from pathlib import Path
import pandas as pd
import yaml
import lightgbm as lgbm
from pathlib import Path
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from src.boruta import BorutaPy
import warnings
warnings.filterwarnings('ignore')

### 1. Load data

In [8]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

model_parameters = config["model_parameters"]["xgbm"]
numeric_features = config["features"]["numerical"]
features = numeric_features
target = config["main"]["target"]
data_train_path = Path.cwd().parent / config["main"]["data_train_path"]
train_validation_path = Path.cwd().parent / config["main"]["data_validation_path"]

train_df = pd.read_pickle(data_train_path)
validation_df = pd.read_pickle(train_validation_path)

X_train, Y_train = train_df[features], train_df[target]
X_val, Y_val = validation_df[features], validation_df[target]

split_seed = config["main"]["random_seed"]

### 1. Boruta

In [9]:
model = XGBClassifier(**model_parameters, random_state=split_seed)   

bfs = BorutaPy(model, n_estimators='auto', random_state=split_seed).fit(
    train_df[features], train_df[target]
)
boruta_features = list(np.array(features)[bfs.support_])
boruta_features

['previous_internal_apps__account_to_application_days',
 'previous_internal_apps__n_bnpl_approved_apps',
 'previous_internal_apps__n_sf_apps',
 'previous_internal_apps__ratio_bnpl_approved',
 'credit_reports__balance_due_ratio_median',
 'credit_reports__balance_due_std_revolvente',
 'credit_reports__balance_due_worst_delay_ratio_median_pagos_fijos',
 'credit_reports__cdc_inquiry_id_count_por_determinar',
 'credit_reports__credit_limit_median_revolvente',
 'credit_reports__debt_ratio_median_pagos_fijos',
 'credit_reports__loans_with_at_least_one_delayed_ratio',
 'credit_reports__severity_delayed_payments_median',
 'credit_reports__severity_delayed_payments_median_pagos_fijos',
 'credit_reports__severity_delayed_payments_median_revolvente']